# this notebook parse patch data
# TODO: meet with Venu to confirm the changes I made
# TODO: figure out the units in COSMED

In [1]:
import numpy as np
np.random.seed(0) 

import argparse

import os
import math
from math import sin

import json

import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import seaborn as sns

import matplotlib
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')
matplotlib.rc( 'savefig', facecolor = 'white' )
from matplotlib import pyplot
%matplotlib inline

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms, datasets, models
from torchsummary import summary
torch.manual_seed(0)

i_seed = 0

from datetime import datetime
import pytz
import pprint

import sys
sys.path.append('../') # add this line so Data and data are visible in this file
sys.path.append('../../') # add this line so Data and data are visible in this file
sys.path.append('../PatchWand/') # add this line so Data and data are visible in this file

# from PatchWand import *
from plotting_tools import *
from setting import *
from models import *
from models_CNN import *
from models_CNN2 import *
from models_resnet import *
from evaluate import *

from stage3_preprocess import *
from stage4_regression import *
from training_util import *
from dataset_util import *
from dataIO import *

from importlib import reload
%load_ext autoreload
%autoreload 2

In [2]:
print(torch.version.cuda)

11.3


In [3]:
parser = argparse.ArgumentParser(description='SpO2_estimate')
parser.add_argument('--input_folder', metavar='input_folder', help='input_folder',
                    default='../')
parser.add_argument('--output_folder', metavar='output_folder', help='output_folder',
                    default='../')
parser.add_argument('--subject_id', metavar='subject_id', help='subject_id',
                    default='101')
parser.add_argument('--training_params_file', metavar='training_params_file', help='training_params_file',
                    default='training_params_list.json')


# checklist 3: comment first line, uncomment second line
args = parser.parse_args(['--input_folder', '../../data/stage3/', 
                          '--output_folder', '../../data/stage4/',
#                           '--training_params_file', 'training_params_baseline.json',
                          '--training_params_file', 'training_params_dummy.json',
                         ])
# args = parser.parse_args()
print(args)

Namespace(input_folder='../../data/stage3/', output_folder='../../data/stage4/', subject_id='101', training_params_file='training_params_dummy.json')


# start timing

In [4]:

tz_NY = pytz.timezone('America/New_York') 
datetime_start = datetime.now(tz_NY)
print("start time:", datetime_start.strftime("%Y-%b-%d %H:%M:%S"))



start time: 2022-Mar-15 23:47:39


In [5]:
inputdir = args.input_folder
outputdir = args.output_folder
training_params_file = args.training_params_file

if not os.path.exists(outputdir):
    os.makedirs(outputdir)


# get training params and dataloaders

In [6]:
torch.cuda.is_available()

True

In [7]:
with open(training_params_file) as json_file:
    training_params_list = json.load(json_file)

for training_params in [training_params_list[0]]:
    # include device in training_params
    device = torch.device('cuda:{}'.format(int(training_params['cuda_i'])) if torch.cuda.is_available() else 'cpu')
    training_params['device'] = device


    training_params['CV_config'] = {
        'subject_id': 113,
        'task_id': 5,
    }
    stage3_dict = data_loader('stage3_dict', inputdir).item()
    training_params['list_signal'] = stage3_dict['list_signal']
    training_params['list_feature'] = stage3_dict['list_feature']
    training_params['list_output'] = stage3_dict['list_output']
    training_params['list_meta'] = stage3_dict['list_meta']
    training_params['FS_RESAMPLE_DL'] = stage3_dict['FS_RESAMPLE_DL']
    training_params['subject_ids'] = stage3_dict['subject_ids']
    training_params['task_ids'] = stage3_dict['task_ids']
    
    dataloaders, dataset_sizes = get_loaders(inputdir, training_params)
    print('data dimensions are:', dataloaders['train'].dataset.data.shape)

    data_dimensions = dataloaders['train'].dataset.__getitem__(0)[0].size()
    training_params['data_dimensions'] = list(data_dimensions)
    
    sweep_name = training_params['sweep_name'] 
    
    if training_params['model_name'] == 'FeatureExtractor_CNN':
        training_params['featrue_extractor'] = FeatureExtractor_CNN
    elif training_params['model_name'] == 'ResNet1D':
        training_params['featrue_extractor'] = ResNet1D
    elif training_params['model_name'] == 'FeatureExtractor_CNN2':
        training_params['featrue_extractor'] = FeatureExtractor_CNN2




# training_params = training_params_list[0]

data dimensions are: (1909, 3, 6000)


In [8]:
outputdir = outputdir+training_params['model_name']+'/'
if not os.path.exists(outputdir):
    os.makedirs(outputdir)
    

In [9]:
training_params

{'model_name': 'ResNet1D',
 'sweep_name': 'baseline',
 'kernel_size': [5],
 'channel_n': 4,
 'num_epochs': 5,
 'batch_size': 64,
 'learning_rate': 0.001,
 'cuda_i': 1,
 'use_sc': True,
 'fusion_type': 'late',
 'pooling_type': 'avg_pooling',
 'stride': [2],
 'groups': [1],
 'downsample_gap': [2],
 'n_block_macro': [2],
 'wandb': True,
 'input_names': ['ECG', 'accelZ', 'ppg_g_1'],
 'feature_names': ['weight', 'height', 'gender'],
 'output_names': ['EE_cosmed', 'RR_cosmed'],
 'loss_weights': {'EE_cosmed': 1, 'RR_cosmed': 0.5},
 'meta_names': ['subject_id', 'task'],
 'device': device(type='cuda', index=1),
 'CV_config': {'subject_id': 113, 'task_id': 5},
 'list_signal': ['ECG', 'accelZ', 'ppg_g_1'],
 'list_feature': ['VE_cosmed',
  'HR_patch',
  'weight',
  'height',
  'gender',
  'age',
  'BMR'],
 'list_output': ['RR_cosmed',
  'VT_cosmed',
  'EE_cosmed',
  'SPO2_cosmed',
  'HR_cosmed',
  'VO2_cosmed',
  'resp_cosmed'],
 'list_meta': ['subject_id', 'task'],
 'FS_RESAMPLE_DL': 100,
 'subje

In [10]:
import wandb

# training_params['wandb'] = True

if training_params['wandb']:
    wandb.login()


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: mchan81 (use `wandb login --relogin` to force relogin)


In [25]:
def train_sweep(config=None):

#     with wandb.init(config=config, entity='inanlab', project="[TL] stage2_cnn", reinit=True, dir=outputdir):
    with wandb.init(config=config, reinit=True, dir=outputdir):

        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config
        
        print(config)
#         print(training_params)
        sys.exit()
        
        # init the model
        
        training_params['kernel_size'] = config['kernel_size']
        training_params['kernel_size'] = config['kernel_size']
        
        model = resp_multiverse(training_params=training_params)

        
        

        ordered_subject_ids = np.asarray([115, 107, 113, 110, 101, 104, 106, 121, 212, 102, 103, 111, 114, 116, 118, 119, 120])



        # for subject_id in training_params['subject_ids']:
        for subject_id in ordered_subject_ids:

            config['CV_config']['subject_id'] = subject_id

            device = torch.device('cuda:{}'.format(int(training_params['cuda_i'])) if torch.cuda.is_available() else 'cpu')
            print('using device', device)


            print('using model ', training_params['model_name'])

            model = resp_multiverse(training_params=training_params)

            model = model.to(device).float()

            optimizer = torch.optim.Adam(model.parameters(), lr=training_params['learning_rate'], weight_decay=0.01)

            training_params['criterion'] = criterion
            training_params['optimizer'] = optimizer
            training_params['inputdir'] = inputdir

            CV_dict = train_model(model, training_params, trainer, evaler, preder)
#             plot_losses(CV_dict, outputdir=outputdir, show_plot=False)

#             for task in training_params['tasks']:
#                 label_est_val = CV_dict['performance_dict_val']['out_dict'][task]
#                 label_val = CV_dict['performance_dict_val']['label_dict'][task]

#                 label_est_train = CV_dict['performance_dict_train']['out_dict'][task]
#                 label_train = CV_dict['performance_dict_train']['label_dict'][task]

#                 # get performance df for training and testing dataset
#                 df_performance_train[task] = df_performance_train[task].append( get_df_performance(label_train, label_est_train, subject_id, task), ignore_index=True )
#                 df_performance_train[task].to_csv(outputdir+'df_performance_train_{}.csv'.format(task), index=False)

#                 df_outputlabel_train[task] = df_outputlabel_train[task].append(
#                     pd.DataFrame( {
#                     'label_est': label_est_train,
#                     'label': label_train,
#                     'CV': [subject_id]*label_train.shape[0],
#                     'task': [task]*label_train.shape[0]
#                     }), ignore_index=True )

#                 df_outputlabel_train[task].to_csv(outputdir+'df_outputlabel_train_{}.csv'.format(task), index=False)

#                 df_performance_val[task] = df_performance_val[task].append( get_df_performance(label_val, label_est_val, subject_id, task), ignore_index=True )
#                 df_performance_val[task].to_csv(outputdir+'df_performance_val_{}.csv'.format(task), index=False)

#                 df_outputlabel_val[task] = df_outputlabel_val[task].append(
#                     pd.DataFrame( {
#                     'label_est': label_est_val,
#                     'label': label_val,
#                     'CV': [subject_id]*label_val.shape[0],
#                     'task': [task]*label_val.shape[0]
#                     }), ignore_index=True )

#                 df_outputlabel_val[task].to_csv(outputdir+'df_outputlabel_val_{}.csv'.format(task), index=False)

#                 # plot performance training and testing dataset
#                 plot_regression(df_outputlabel_train[task], df_performance_train[task], task, fig_name='regression_train_{}'.format(task), show_plot=False, outputdir=outputdir+'model_output/')
#                 plot_BA(df_outputlabel_train[task], task, fig_name='BA_train_{}'.format(task), show_plot=False, outputdir=outputdir+'model_output/')

#                 plot_regression(df_outputlabel_val[task], df_performance_val[task], task, fig_name='regression_val_{}'.format(task), show_plot=False, outputdir=outputdir+'model_output/')
#                 plot_BA(df_outputlabel_val[task], task, fig_name='BA_val_{}'.format(task), show_plot=False, outputdir=outputdir+'model_output/')

#                 plot_output(df_outputlabel_train[task], task, fig_name = 'outputINtime_train_', show_plot=True, outputdir=outputdir+'model_output/')
#                 plot_output(df_outputlabel_val[task], task, fig_name = 'outputINtime_val_',  show_plot=True, outputdir=outputdir+'model_output/')


#                 check_featuremap(model, dataloaders['val'], training_params, fig_name = 'DL_activation_{}_'.format(subject_id), outputdir=outputdir+'activation_layers/', show_plot=False)




#                 print('rmse={:.2f}, mae={:.2f}, PCC={:.2f}'.format(df_performance_val['EE_cosmed']['rmse'].mean(), df_performance_val['EE_cosmed']['mae'].mean(), df_performance_val['EE_cosmed']['PCC'].mean()))




        
        
#     #         model = train_model(model, dataloaders, config)
#         model, performance_dict_train, performance_dict_val = train_model(model, dataloaders, config,  trainer, evaler, preder)
    

        
# #         wandb.log({'model_out_train': wandb.Image(plot_model_output(performance_dict_train))})
#         wandb.log({'model_out_train': plot_model_output(performance_dict_train, 'model_output_train')})
#         wandb.log({'model_out_val': plot_model_output(performance_dict_val, 'model_output_val')})

        

    
    
#         title_str = 'confusion matrxi for classifier {}\n'.format('resnet (train)')
#         PIL_train = aggregate_performance_dict(performance_dict_train, config, title_str)
#         wandb.log({'cm_train': wandb.Image(PIL_train)})

#         fig_roc_train = plot_ROC(performance_dict_train)
#         wandb.log({'roc_train': wandb.Image(fig_roc_train)})
    
    
#         title_str = 'confusion matrxi for classifier {}\n'.format('resnet (val)')
#         PIL_val = aggregate_performance_dict(performance_dict_val, config, title_str)
#         wandb.log({'cm_val': wandb.Image(PIL_val)})
        
#         fig_roc_val = plot_ROC(performance_dict_val)
#         wandb.log({'roc_val': wandb.Image(fig_roc_val)})


In [30]:
sweep_config = {
#     'program': '{}.py'.format(training_params['arch']),
    'program': 'resnet.py',
    'method': 'random',
    'metric': {
        'goal': 'maximize',
        'name': 'val_acc',},
    "parameters" : {
        "epochs" : {
            "values" : [10, 20, 50]
        },
        "learning_rate" :{
            "min": 0.0001,
            "max": 0.1
        }
    }
}



In [14]:
def is_number(a):
    # will be True also for 'NaN'
    try:
        number = float(a)
        return True
    except TypeError:
        return False

In [22]:
# should be numbers or boolean
parameters_dict = {}

for key, value in training_params.items():
    print(value)
    parameters_dict[key] = {
         'values': value
    }
#     if key in ['device', 'data_dimensions', 'sweep_name', 'arch']:
#         continue
#         parameters_dict[key] = {
#             'value': value
#         }
#     else:
#     if is_number(value):
#         parameters_dict[key] = {
#             'value': value
#         }
#     else:
#         parameters_dict[key] = {
#             'values': value
#         }


# sweep_config['parameters'] = parameters_dict

ResNet1D
baseline
[5]
4
5
64
0.001
1
True
late
avg_pooling
[2]
[1]
[2]
[2]
True
['ECG', 'accelZ', 'ppg_g_1']
['weight', 'height', 'gender']
['EE_cosmed', 'RR_cosmed']
{'EE_cosmed': 1, 'RR_cosmed': 0.5}
['subject_id', 'task']
cuda:1
{'subject_id': 113, 'task_id': 5}
['ECG', 'accelZ', 'ppg_g_1']
['VE_cosmed', 'HR_patch', 'weight', 'height', 'gender', 'age', 'BMR']
['RR_cosmed', 'VT_cosmed', 'EE_cosmed', 'SPO2_cosmed', 'HR_cosmed', 'VO2_cosmed', 'resp_cosmed']
['subject_id', 'task']
100
[101 102 103 104 106 107 110 111 113 114 115 116 118 119 120 121 212]
[0 1 2 3 4 5]
[3, 6000]
<class 'models_resnet.ResNet1D'>


In [31]:
if training_params['wandb']:
    print('sweeping for:', sweep_name)
    
#     with wandb.init(config=config, entity='inanlab', project="[TL] stage2_cnn", reinit=True, dir=outputdir):
    sweep_id = wandb.sweep(sweep_config, entity='inanlab', project='[EE] stage4_'+training_params['sweep_name'])

#     sweep_id = wandb.sweep(sweep_config, project=sweep_name)
    wandb.agent(sweep_id, train_sweep)

sweeping for: baseline
Create sweep with ID: bn3hs43o
Sweep URL: https://wandb.ai/inanlab/%5BEE%5D%20stage4_baseline/sweeps/bn3hs43o


wandb: Agent Starting Run: 7zy8lfzu with config:
wandb: 	epochs: 20
wandb: 	learning_rate: 0.0861497266574595
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: wandb version 0.12.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


{'epochs': 20, 'learning_rate': 0.0861497266574595}


wandb: Agent Starting Run: prv3bmml with config:
wandb: 	epochs: 10
wandb: 	learning_rate: 0.06763614398903549
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: wandb version 0.12.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


{'epochs': 10, 'learning_rate': 0.06763614398903549}


wandb: Agent Starting Run: mywoza2z with config:
wandb: 	epochs: 50
wandb: 	learning_rate: 0.004307631342029089
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: wandb version 0.12.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


{'epochs': 50, 'learning_rate': 0.004307631342029089}


wandb: Agent Starting Run: yfdjxwxu with config:
wandb: 	epochs: 10
wandb: 	learning_rate: 0.08748504655852164
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: wandb version 0.12.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Ctrl + C detected. Stopping sweep.


Problem at: <ipython-input-25-544eb4338fb6> 4 train_sweep


Traceback (most recent call last):
  File "/home/mchan/miniconda3/envs/mienv/lib/python3.7/site-packages/wandb/sdk/wandb_init.py", line 762, in init
    run = wi.init()
  File "/home/mchan/miniconda3/envs/mienv/lib/python3.7/site-packages/wandb/sdk/wandb_init.py", line 533, in init
    _ = backend.interface.communicate_run_start(run_obj)
  File "/home/mchan/miniconda3/envs/mienv/lib/python3.7/site-packages/wandb/sdk/interface/interface.py", line 806, in communicate_run_start
    result = self._communicate(rec)
  File "/home/mchan/miniconda3/envs/mienv/lib/python3.7/site-packages/wandb/sdk/interface/interface.py", line 544, in _communicate
    return self._communicate_async(rec, local=local).get(timeout=timeout)
  File "/home/mchan/miniconda3/envs/mienv/lib/python3.7/site-packages/wandb/sdk/interface/interface.py", line 82, in get
    is_set = self._object_ready.wait(timeout)
  File "/home/mchan/miniconda3/envs/mienv/lib/python3.7/threading.py", line 552, in wait
    signaled = self._co

In [20]:
training_params['sweep_name']

'baseline'

{'EE_cosmed': 1, 'RR_cosmed': 0.5}

# define multi-task loss

In [14]:
training_params['tasks'] = training_params['output_names']

training_params['criterions'] = {}
# training_params['loss_weights'] = {}

for task in training_params['tasks']:
    training_params['criterions'][task] =  torch.nn.MSELoss()
    
#     if task == 'EE_cosmed':
#         training_params['loss_weights'][task] =  1
#     else:
#         training_params['loss_weights'][task] =  0.1

criterion = MultiTaskLoss(training_params)

# some random param
## kernel_size = 5 ~ 16*100/300 (16 for 300 Hz sampling rate, 5 for 100 sampling rate)


In [14]:
# training_params['downsample_gap']

# # n_block = training_params['n_block_macro'][0] * training_params['downsample_gap'][0]
# # downsample_gap = training_params['downsample_gap'][0]
# # base_filters = training_params['base_filters'][0]
# # use_sc =  training_params['use_sc']
# # increasefilter_gap = downsample_gap * 2
# # kernel_size = training_params['kernel_size']
# # groups = training_params['base_filters'][0]

In [15]:
if training_params['model_name'] == 'ResNet1D':
# reference: https://github.com/hsd1503/resnet1d/blob/master/test_physionet.py
    training_params['n_block'] = [training_params['n_block_macro'][0] * training_params['downsample_gap'][0]]
    training_params['base_filters'] = [training_params['channel_n']] # [64] 
    training_params['in_channels'] = training_params['data_dimensions'][0]
#     training_params['increasefilter_gap'] = [training_params['downsample_gap'][0] * 2]
    training_params['increasefilter_gap'] = [training_params['downsample_gap'][0]]
    

In [16]:
# # kernel_size = 10 # ~ 16*100/300 (16 for 300 Hz sampling rate, 5 for 100 sampling rate)

# training_params['stride'] = 2
# # stride = 2
# # increasefilter_gap = 12
# # base_filters = 32
# training_params['n_classes'] = 1
# # n_classes = 1 # regression
# training_params['groups'] = 32
# # groups = 32


In [17]:
training_params['data_dimensions']

[3, 6000]

# test the model

# TODO: need to fix summary()

In [19]:
def test_model(training_params):

    print('using model ', training_params['model_name'])

    model = resp_multiverse(training_params=training_params)
    summary(model, input_size=[tuple(training_params['data_dimensions']), (model.N_features,1)], device='cpu')
    del model


debug_model = True
if debug_model==True:
    test_model(training_params)

using model  ResNet1D
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1              [-1, 4, 6000]              24
   MyConv1dPadSame-2              [-1, 4, 6000]               0
       BatchNorm1d-3              [-1, 4, 6000]               8
              ReLU-4              [-1, 4, 6000]               0
            Conv1d-5              [-1, 4, 6000]              84
   MyConv1dPadSame-6              [-1, 4, 6000]               0
       BatchNorm1d-7              [-1, 4, 6000]               8
              ReLU-8              [-1, 4, 6000]               0
           Dropout-9              [-1, 4, 6000]               0
           Conv1d-10              [-1, 4, 6000]              84
  MyConv1dPadSame-11              [-1, 4, 6000]               0
       BasicBlock-12              [-1, 4, 6000]               0
      BatchNorm1d-13              [-1, 4, 6000]               8
             ReLU

# define training, validating, and evaluating funcitons 

In [20]:
trainer = train_resnet
evaler = eval_resnet
preder = pred_resnet

# plot the input signals

In [21]:
# k = 90

# plt.plot(dataloaders['train'].dataset.data[k,0,:])
# plt.plot(dataloaders['train'].dataset.data[k,1,:])
# plt.plot(dataloaders['train'].dataset.data[k,2,:])

# training the model

In [22]:
# def train_model(model, training_params, trainer, evaler, preder):

#     inputdir = training_params['inputdir']
    
#     dataloaders, dataset_sizes = get_loaders(inputdir, training_params)

#     total_loss_train = np.zeros(training_params['num_epochs'])
#     total_loss_val = np.zeros(training_params['num_epochs'])

#     print('\t start training.....')

#     for epoch in range(training_params['num_epochs']):
#         if epoch%10==1:
#             print(epoch)
#         training_params['epoch'] = epoch

#         ##### model training mode ####
#         performance_dict_train = trainer(model, dataloaders['train'], training_params)
#         total_loss_train[epoch] = performance_dict_train['total_loss']

#         performance_dict_val = evaler(model, dataloaders['val'], training_params)
#         total_loss_val[epoch] = performance_dict_val['total_loss']

#     print('\t done with training.....')

#     performance_dict_train = preder(model, dataloaders['train'], training_params)
#     performance_dict_val = preder(model, dataloaders['val'], training_params)

    
#     CV_dict = {
#         'performance_dict_train': performance_dict_train,
#         'total_loss_train': total_loss_train,
#         'performance_dict_val': performance_dict_val,
#         'total_loss_val': total_loss_val,
#         'model': model,
#         'subject_id_val': training_params['CV_config']['subject_id'], 
#     }
    
#     return CV_dict

In [17]:
pprint.pprint(training_params)

{'CV_config': {'subject_id': 113, 'task_id': 5},
 'FS_RESAMPLE_DL': 100,
 'batch_size': 64,
 'channel_n': 4,
 'criterions': {'EE_cosmed': MSELoss(), 'RR_cosmed': MSELoss()},
 'cuda_i': 1,
 'data_dimensions': [3, 6000],
 'device': device(type='cuda', index=1),
 'downsample_gap': [2],
 'featrue_extractor': <class 'models_resnet.ResNet1D'>,
 'feature_names': ['weight', 'height', 'gender'],
 'fusion_type': 'late',
 'groups': [1],
 'input_names': ['ECG', 'accelZ', 'ppg_g_1'],
 'kernel_size': [5],
 'learning_rate': 0.001,
 'list_feature': ['VE_cosmed',
                  'HR_patch',
                  'weight',
                  'height',
                  'gender',
                  'age',
                  'BMR'],
 'list_meta': ['subject_id', 'task'],
 'list_output': ['RR_cosmed',
                 'VT_cosmed',
                 'EE_cosmed',
                 'SPO2_cosmed',
                 'HR_cosmed',
                 'VO2_cosmed',
                 'resp_cosmed'],
 'list_signal': ['ECG', 'acc

In [26]:
def check_featuremap(model, dataloader, training_params, fig_name=None, show_plot=False, outputdir=None):

    # 1. set up the hook
    activation = {}
    def get_activation(name):
        def hook(model, input, output):
            activation[name] = output.detach()
        return hook

    data = torch.from_numpy(dataloader.dataset.data)
    feature = torch.from_numpy(dataloader.dataset.feature)
    data = data.to(device).float()
    feature = feature.to(device).float()

    # 2. check one sample only
    N_samples = dataloader.dataset.data.shape[0]
    np.random.seed(0)
    i_sample = np.random.randint(N_samples)

    #     model_hooking(model, training_params)

    model_name = training_params['model_name']

    # 3. define the layers that I want to look at
    if model_name=='FeatureExtractor_CNN2':
        key = list(model.feature_extractors.keys())[0]
        model.feature_extractors[key].layer1.register_forward_hook(get_activation('layer1'))
        model.feature_extractors[key].layer2.register_forward_hook(get_activation('layer2'))
        model.feature_extractors[key].layer3.register_forward_hook(get_activation('layer3'))
        model.feature_extractors[key].layer4.register_forward_hook(get_activation('layer4'))
        model.regressors.EE_cosmed.fc1.register_forward_hook(get_activation('fc1'))
        model.regressors.EE_cosmed.fc2.register_forward_hook(get_activation('fc2'))

        layer_names = ['layer1', 'layer2', 'layer3', 'layer4', 'fc1', 'fc2']

    elif model_name=='FeatureExtractor_CNN':
        key = list(model.feature_extractors.keys())[0]
        model.feature_extractors[key].layer1.register_forward_hook(get_activation('layer1'))
        model.feature_extractors[key].layer2.register_forward_hook(get_activation('layer2'))
        model.feature_extractors[key].layer3.register_forward_hook(get_activation('layer3'))
        model.feature_extractors[key].layer4.register_forward_hook(get_activation('layer4'))
        model.regressors.EE_cosmed.fc1.register_forward_hook(get_activation('fc1'))
        model.regressors.EE_cosmed.fc2.register_forward_hook(get_activation('fc2'))
        layer_names = ['layer1', 'layer2', 'layer3', 'layer4', 'fc1', 'fc2']

    if model_name=='ResNet1D':
        
        
        model.feature_extractors.ECG.basicblock_list[-1].register_forward_hook(get_activation('ecg_layer_last'))
#         model.feature_extractors.ECG.basicblock_list[4].register_forward_hook(get_activation('ecg_layer5'))
#         model.feature_extractors.ECG.basicblock_list[8].register_forward_hook(get_activation('ecg_layer3'))
#         model.feature_extractors.ECG.basicblock_list[12].register_forward_hook(get_activation('ecg_layer13'))
        model.feature_extractors.accelZ.basicblock_list[-1].register_forward_hook(get_activation('scg_layer_last'))
#         model.feature_extractors.accelZ.basicblock_list[4].register_forward_hook(get_activation('scg_layer5'))
#         model.feature_extractors.accelZ.basicblock_list[8].register_forward_hook(get_activation('scg_layer3'))
#         model.feature_extractors.accelZ.basicblock_list[12].register_forward_hook(get_activation('scg_layer13'))
        model.feature_extractors.ppg_g_1.basicblock_list[-1].register_forward_hook(get_activation('ppg_layer_last'))
#         model.feature_extractors.ppg_g_1.basicblock_list[4].register_forward_hook(get_activation('ppg_layer5'))
#         model.feature_extractors.ppg_g_1.basicblock_list[8].register_forward_hook(get_activation('ppg_layer3'))
#         model.feature_extractors.ppg_g_1.basicblock_list[12].register_forward_hook(get_activation('ppg_layer13'))
    

#         model.regressors.EE_cosmed.fc1.register_forward_hook(get_activation('fc1'))
#         model.regressors.EE_cosmed.fc2.register_forward_hook(get_activation('fc2'))
        
        
#         layer_names = ['ecg_layer1', 'ecg_layer5', 'ecg_layer13', 'scg_layer1', 'scg_layer5', 'scg_layer13', 'pcg_layer1', 'ppg_layer5', 'ppg_layer13', 'fc1', 'fc2']
        layer_names = ['ecg_layer_last', 'scg_layer_last', 'ppg_layer_last']

    # 4. pass the data to the model and the hook will take care of the rest (output stored in activation)
#     output = model(data, feature)
    _ = model(data, feature)

    data = data.cpu().detach().numpy()

    # 5. organize these activation layers
    data_layers = {}
    for layer_name in layer_names:
        data_layers[layer_name] = activation[layer_name].cpu().detach().numpy()


    for layer_name in layer_names:
        
        data_layer = data_layers[layer_name]
#         print(data_layer.shape)

        N_sigs = len(training_params['input_names'])

        fig, axes = plt.subplots(data_layer.shape[1]+1,1, figsize=(15, data_layer.shape[1]), dpi=80, gridspec_kw = {'wspace':0, 'hspace':0}, facecolor='white')
        fontsize = 13

        FS_RESAMPLE_DL = training_params['FS_RESAMPLE_DL']
        t_arr = np.arange(data.shape[-1])/FS_RESAMPLE_DL

#         for i, input_name in enumerate(training_params['input_names']):

        # 1. plot one physio sig at the top column
        ax = axes[0]

        if 'ecg' in layer_name:
            unit = unit_dict['ecg']
            i_sig = training_params['input_names'].index('ECG')
        elif 'scg' in layer_name:
            unit = unit_dict['accel']
            i_sig = training_params['input_names'].index('accelZ')
        elif 'ppg' in layer_name:
            unit = unit_dict['ppg']
            i_sig = training_params['input_names'].index('ppg_g_1')

        sig_name = training_params['input_names'][i_sig]
        ax.plot(t_arr, data[i_sample,i_sig,:])
        ax.set_xlim(t_arr.min(), t_arr.max()) # remove the weird white space at the beg and end of the plot
        ax.set_ylabel('{}\n[{}]'.format(sig_name, unit), fontsize=fontsize,rotation = 0,  va='center', ha='center',  labelpad=100)

        # 2. next, plot the feature map
        t_arr = np.arange(data_layer.shape[-1]) / FS_RESAMPLE_DL / (2**(training_params['n_block_macro'][0]))
        for j_filter, ax in enumerate(axes[1:]):

            if j_filter!=len(axes[1:])-1:
                ax.set_xticklabels([])

            ax.plot(t_arr, data_layer[i_sample, j_filter, :].T, alpha=1)

            ax.set_xlim(t_arr.min(), t_arr.max()) # remove the weird white space at the beg and end of the plot

            # remove some borders (top and right)
            ax.spines['right'].set_visible(False)
            if j_filter==0:
                ax.spines['top'].set_visible(False)

            ax.set_ylabel('filter {}'.format(j_filter), fontsize=fontsize,rotation = 0,  va='center', ha='center',  labelpad=100)
            # set tick font size
            ax.tick_params(axis='both', which='major', labelsize=fontsize*0.8)

            ax.set_xlabel('time (sec)', fontsize=fontsize)
            fig.subplots_adjust(wspace=0, hspace=0)

        fig.tight_layout()
        
        if fig_name is None:
            fig_name = 'DL_activation_'
#             print('hihi', sig_name)
#         fig_name = 'DL_activation_'+sig_name

#         print(fig_name)

        if outputdir is not None:
            if not os.path.exists(outputdir):
                os.makedirs(outputdir)
            fig.savefig(outputdir + fig_name+sig_name + '.png', facecolor=fig.get_facecolor())

        if show_plot == False:
            plt.close(fig)
            pyplot.close(fig)
            plt.close('all')
            
        

#     fig, ax_last = plt.subplots(1,1, figsize=(20, 3), dpi=80)
#     ax_last.plot(data_fc1[i_sample, :], alpha=1)


In [26]:
df_performance_train = {}
df_performance_val = {}

df_outputlabel_train = {}
df_outputlabel_val = {}

for task in training_params['tasks']:

    df_performance_train[task] = pd.DataFrame()
    df_performance_val[task] = pd.DataFrame()

    df_outputlabel_train[task] = pd.DataFrame()
    df_outputlabel_val[task] = pd.DataFrame()

In [27]:

ordered_subject_ids = np.asarray([115, 107, 113, 110, 101, 104, 106, 121, 212, 102, 103, 111, 114, 116, 118, 119, 120])



# for subject_id in training_params['subject_ids']:
for subject_id in ordered_subject_ids:

    training_params['CV_config']['subject_id'] = subject_id

    device = torch.device('cuda:{}'.format(int(training_params['cuda_i'])) if torch.cuda.is_available() else 'cpu')
    print('using device', device)

    
    print('using model ', training_params['model_name'])

    model = resp_multiverse(training_params=training_params)

    model = model.to(device).float()

    optimizer = torch.optim.Adam(model.parameters(), lr=training_params['learning_rate'], weight_decay=0.01)

    training_params['criterion'] = criterion
    training_params['optimizer'] = optimizer
    training_params['inputdir'] = inputdir

    CV_dict = train_model(model, training_params, trainer, evaler, preder)
    plot_losses(CV_dict, outputdir=outputdir, show_plot=False)

    for task in training_params['tasks']:
        label_est_val = CV_dict['performance_dict_val']['out_dict'][task]
        label_val = CV_dict['performance_dict_val']['label_dict'][task]

        label_est_train = CV_dict['performance_dict_train']['out_dict'][task]
        label_train = CV_dict['performance_dict_train']['label_dict'][task]
        
        # get performance df for training and testing dataset
        df_performance_train[task] = df_performance_train[task].append( get_df_performance(label_train, label_est_train, subject_id, task), ignore_index=True )
        df_performance_train[task].to_csv(outputdir+'df_performance_train_{}.csv'.format(task), index=False)

        df_outputlabel_train[task] = df_outputlabel_train[task].append(
            pd.DataFrame( {
            'label_est': label_est_train,
            'label': label_train,
            'CV': [subject_id]*label_train.shape[0],
            'task': [task]*label_train.shape[0]
            }), ignore_index=True )
        
        df_outputlabel_train[task].to_csv(outputdir+'df_outputlabel_train_{}.csv'.format(task), index=False)

        df_performance_val[task] = df_performance_val[task].append( get_df_performance(label_val, label_est_val, subject_id, task), ignore_index=True )
        df_performance_val[task].to_csv(outputdir+'df_performance_val_{}.csv'.format(task), index=False)
        
        df_outputlabel_val[task] = df_outputlabel_val[task].append(
            pd.DataFrame( {
            'label_est': label_est_val,
            'label': label_val,
            'CV': [subject_id]*label_val.shape[0],
            'task': [task]*label_val.shape[0]
            }), ignore_index=True )
        
        df_outputlabel_val[task].to_csv(outputdir+'df_outputlabel_val_{}.csv'.format(task), index=False)

        # plot performance training and testing dataset
        plot_regression(df_outputlabel_train[task], df_performance_train[task], task, fig_name='regression_train_{}'.format(task), show_plot=False, outputdir=outputdir+'model_output/')
        plot_BA(df_outputlabel_train[task], task, fig_name='BA_train_{}'.format(task), show_plot=False, outputdir=outputdir+'model_output/')
        
        plot_regression(df_outputlabel_val[task], df_performance_val[task], task, fig_name='regression_val_{}'.format(task), show_plot=False, outputdir=outputdir+'model_output/')
        plot_BA(df_outputlabel_val[task], task, fig_name='BA_val_{}'.format(task), show_plot=False, outputdir=outputdir+'model_output/')

        plot_output(df_outputlabel_train[task], task, fig_name = 'outputINtime_train_', show_plot=True, outputdir=outputdir+'model_output/')
        plot_output(df_outputlabel_val[task], task, fig_name = 'outputINtime_val_',  show_plot=True, outputdir=outputdir+'model_output/')

        
    check_featuremap(model, dataloaders['val'], training_params, fig_name = 'DL_activation_{}_'.format(subject_id), outputdir=outputdir+'activation_layers/', show_plot=False)


#     sys.exit()
    del model

print('rmse={:.2f}, mae={:.2f}, PCC={:.2f}'.format(df_performance_val['EE_cosmed']['rmse'].mean(), df_performance_val['EE_cosmed']['mae'].mean(), df_performance_val['EE_cosmed']['PCC'].mean()))


using device cuda:1
using model  ResNet1D
	 start training.....
1
	 done with training.....
using device cuda:1
using model  ResNet1D
	 start training.....
1
	 done with training.....
using device cuda:1
using model  ResNet1D
	 start training.....
1
	 done with training.....
using device cuda:1
using model  ResNet1D
	 start training.....


KeyboardInterrupt: 

In [ ]:
# end timing

In [1]:

datetime_end = datetime.now(tz_NY)
print("end time:", datetime_end.strftime("%Y-%b-%d %H:%M:%S"))

duration = datetime_end-datetime_start
duration_in_s = duration.total_seconds()
days    = divmod(duration_in_s, 86400)        # Get days (without [0]!)
hours   = divmod(days[1], 3600)               # Use remainder of days to calc hours
minutes = divmod(hours[1], 60)                # Use remainder of hours to calc minutes
seconds = divmod(minutes[1], 1)               # Use remainder of minutes to calc seconds
print("Time between dates: %d days, %d hours, %d minutes and %d seconds" % (days[0], hours[0], minutes[0], seconds[0]))

NameError: name 'datetime' is not defined

In [ ]:
sys.exit()

In [ ]:
training_params['model_name']

In [ ]:
# # def get_activation(name):
# #     activation = {}
# #     def hook(model, input, output):
# #         activation[name] = output.detach()
# #     return hook
    
# def model_hooking(model, training_params, get_activation):

#     model_name = training_params['model_name']
    
#     if model_name=='FeatureExtractor_CNN2':
#         key = list(model.feature_extractors.keys())[0]
#         model.feature_extractors[key].layer1.register_forward_hook(get_activation('layer1'))
#         model.feature_extractors[key].layer2.register_forward_hook(get_activation('layer2'))
#         model.feature_extractors[key].layer3.register_forward_hook(get_activation('layer3'))
#         model.feature_extractors[key].layer4.register_forward_hook(get_activation('layer4'))
#         model.regressors.EE_cosmed.fc1.register_forward_hook(get_activation('fc1'))
#         model.regressors.EE_cosmed.fc2.register_forward_hook(get_activation('fc2'))
        
# #         layer_names = ['layer1', ]
        
#     elif model_name=='FeatureExtractor_CNN':
#         key = list(model.feature_extractors.keys())[0]
#         model.feature_extractors[key].layer1.register_forward_hook(get_activation('layer1'))
#         model.feature_extractors[key].layer2.register_forward_hook(get_activation('layer2'))
#         model.feature_extractors[key].layer3.register_forward_hook(get_activation('layer3'))
#         model.feature_extractors[key].layer4.register_forward_hook(get_activation('layer4'))
#         model.regressors.EE_cosmed.fc1.register_forward_hook(get_activation('fc1'))
#         model.regressors.EE_cosmed.fc2.register_forward_hook(get_activation('fc2'))
#     if model_name=='ResNet1D':
#         pass
# #         key = list(model.feature_extractors.keys())[0]
# #         model.feature_extractors[key].layer1.register_forward_hook(get_activation('layer1'))
# #         model.feature_extractors[key].layer2.register_forward_hook(get_activation('layer2'))
# #         model.feature_extractors[key].layer3.register_forward_hook(get_activation('layer3'))
# #         model.feature_extractors[key].layer4.register_forward_hook(get_activation('layer4'))
# #         model.regressors.EE_cosmed.fc1.register_forward_hook(get_activation('fc1'))
# #         model.regressors.EE_cosmed.fc2.register_forward_hook(get_activation('fc2'))


In [ ]:
# training_params['model_name']

# TODO: improve this block

In [ ]:
performance_dict_train = preder(model, dataloaders['train'], training_params)
performance_dict_val = preder(model, dataloaders['val'], training_params)

unit = unit_dict[task.split('_')[0]]

for task in training_params['tasks']:
    
    
    print('evaluating task:', task)
    MAE, std_AE = get_MAE(performance_dict_train['out_dict'][task], performance_dict_train['label_dict'][task])
    print('\ttrainin: {:.2f}±{:.2f} {}'.format(MAE, std_AE, unit))

    MAE, std_AE = get_MAE(performance_dict_val['out_dict'][task], performance_dict_val['label_dict'][task])
    print('\tval: {:.2f}±{:.2f} {}'.format(MAE, std_AE, unit))



# produce output figures
# TODO: implement the plotting functions below

# TODO!!!!

# for task in tasks
#     plot_loss vs epoch (train and val)
#     plot_MAE, RMSE, vs epoch (train and val)
#     plot scatter plots (show PCC, BD, std, ect.) (just val)
#     plot output vs. label (just val)

In [ ]:
output_name = training_params['output_names'][0].split('_')[0]
unit_dict[output_name]

In [ ]:
# kcalpmin2watt = 69.7333333

# sub_weight = 76

In [ ]:
# performance_dict_train['label_dict']

In [ ]:
training_params['tasks']

In [ ]:
for task in training_params['tasks']:
    fig, (ax1, ax2) = plt.subplots(1,2, figsize=(10,5), dpi=100)
    fontsize = 15
    data_min = np.min(np.r_[performance_dict_train['out_dict'][task], performance_dict_train['label_dict'][task]])
    data_max = np.max(np.r_[performance_dict_train['out_dict'][task], performance_dict_train['label_dict'][task]])
    ax1.scatter(performance_dict_train['out_dict'][task], performance_dict_train['label_dict'][task], alpha=0.3)
    ax1.set_xlim(data_min, data_max)
    ax1.set_ylim(data_min, data_max)
    ax1.plot( [data_min, data_max],[data_min, data_max], '--', color='gray', alpha=0.8)

    ax1.set_xlabel('estimated {} ({})'.format(task.split('_')[0], unit_dict[task.split('_')[0]]), fontsize=fontsize)
    ax1.set_ylabel('true {} ({})'.format(task.split('_')[0], unit_dict[task.split('_')[0]]), fontsize=fontsize)
#     ax.set_xlabel('estimated {} ({})'.format(output_name, 'W'), fontsize=fontsize)
#     ax.set_ylabel('true {} ({})'.format(output_name, 'W'), fontsize=fontsize)

    ax1.set_title('training', fontsize=fontsize)


#     fig, ax = plt.subplots(figsize=(5,5))
#     fontsize = 15
    data_min = np.min(np.r_[performance_dict_val['out_dict'][task], performance_dict_val['label_dict'][task]])
    data_max = np.max(np.r_[performance_dict_val['out_dict'][task], performance_dict_val['label_dict'][task]])
    ax2.scatter(performance_dict_val['out_dict'][task], performance_dict_val['label_dict'][task], alpha=0.3)
    ax2.set_xlim(data_min, data_max)
    ax2.set_ylim(data_min, data_max)
    ax2.plot( [data_min, data_max],[data_min, data_max], '--', color='gray', alpha=0.8)

    ax2.set_xlabel('estimated {} ({})'.format(task.split('_')[0], unit_dict[task.split('_')[0]]), fontsize=fontsize)
    ax2.set_ylabel('true {} ({})'.format(task.split('_')[0], unit_dict[task.split('_')[0]]), fontsize=fontsize)
    
    ax2.set_title('testing', fontsize=fontsize)

#     ax.set_xlabel('estimated {} ({})'.format(output_name, 'W'), fontsize=fontsize)
#     ax.set_ylabel('true {} ({})'.format(output_name, 'W'), fontsize=fontsize)
    plt.show()

